In [1]:
; 练习3.1 ，做一个累加器，做这个是初步认识局部变量
; A是一个过程，那么make-accumulator 返回的是一个过程。

(define (make-accumulator accumulator)
  (lambda (x)                                   ; 返回一个过程
    (begin (set! accumulator (+ x accumulator)) ; 修改这个局部变量
           accumulator )))                      ; 返回这个值

(define A (make-accumulator 5))
(display (A 10))
(newline)
(display (A 10))

15
25

# 用变动的数据做模拟

## 数字电路的模拟器

模拟一个半加器   
![](./图片/3-25半加器.png)  

思路分析：我们希望构造出一个程序，能够模拟我们希望研究的各种数字逻辑电路。这个程序将构造出模拟连线的计算对象，它们能够保持信号，



In [2]:
; 这个模拟中的最基本元素就是连线，过程make-wire,举例来说
(require sicp)
; 如下这个是后来写的，避免错误的
(define (make-wire) nil)
(define (or-gate a b c) nil)
(define (and-gate a b c) nil)
(define (inverter a b) nil)



(define a (make-wire))
(define b (make-wire))
(define c (make-wire))
(define d (make-wire))
(define e (make-wire))
(define s (make-wire))

; 下边是根据连线构造门，比如
(or-gate a b d)
(and-gate a b c)
(inverter c e)

; 然后下边构造一个半加器
(define (half-adder a b s c)
  (let ((d (make-wire)) (e (make-wire)))
    (or-gate a b d)
    (and-gate a b c)
    (inverter c e)
    (and-gate d e s)
    'ok))

; 根据半加器就可以构造出全加器了
(define (full-adder a b c-in sum c-out)
  (let ((s (make-wire))
        (c1 (make-wire))
        (c2 (make-wire)))
    (half-adder b c-in s c1)
    (half-adder a s sum c2)
    (or-gate c1 c2 c-out)
    'ok))

; 然后根据全加器就可以构造出更复杂的电路。
; 以上的根基是连线和各种门，是根据“按愿望思维”先构造出来的，这里就要开吃补充了。
; 我们需要在一根线上的信号变化能够影响到其他连接线上的信号，所以需要如下的操作
; (get-singal <wire>)  : 返回连接线上信号的当前值
; (set-singal! <wire> <new-wire>)  : 修改 
; (add-action! <wire> <procedure of no arguments>)  : 能够将相应连接线上值的变化传递给其他的连线。
; (after-delay )  : 延时的，数字电路肯定是有延时的吧。
; 有了上边这些，就可以定义基本的数字逻辑电路了

(define (inverter input output)
  (define (invert-input)
    (let ((new-value (logical-not (get-signal input))))   ; 取得新值
      (after-delay inverter-delay                         ; 延时若干时间
                   (lambda ()
                     (set-signal! output new-value)))))   ; 更新新值
  (add-action! input invert-input)    ; 为线路关联一个过程，当input改变，就调用invert-input
  'ok)

(define (logical-not s)    ; 这个过程是逻辑反
  (cond ((= s 0) 1)
        ((= s 1) 0)
        (else (error "Invalid signal" s))))

; 如下是与门 
(define (and-gate a1 a2 output)
  (define (and-action-procedure)
    (let ((new-value
           (logical-and (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay
                   (lambda ()
                     (set-signal! output new-value)))))
  ; 如下2个，有一个有变化，就调用 and-action-procedure
  (add-action! a1 and-action-procedure)
  (add-action! a2 and-action-procedure)
  'ok)
; 我写的于运算
(define (logical-and a1 ay)
  (if (and (= a1 1) (= a2 2))
      1
      0
   ))

; 如下是我写的或门
(define (or-gate a1 a2 output)
  (define (or-action-procedure)
    (let ((new-value
           (logical-or (get-signal a1) (get-signal a2))))
      (after-delay and-gate-delay
                   (lambda ()
                     (set-signal! output new-value)))))
  ; 如下2个，有一个有变化，就调用 and-action-procedure
  (add-action! a1 or-action-procedure)
  (add-action! a2 or-action-procedure)
  'ok)

; 我写的或运算
(define (logical-or a1 ay)
  (if (or (= a1 1) (= a2 2))
      1
      0
   ))

; 如下是最基本的线路的表示，

(define (make-wire)
  (let (
        (signal-value 0)                          ; 初始状态
        (action-procedures '()))                  ; 在信号改变时，这些过程都要执行。
    ; 修改新值。
    (define (set-my-signal! new-value)       
      (if (not (= signal-value new-value))        ; 如果状态改变，就执行如下的。
          (begin 
           (set! signal-value new-value)          ; 先改变状态
           (call-each action-procedures))         ; 然后这些过程都要执行。
          'done))
    ; 添加执行过程
    (define (accept-action-procedure! proc)       
      (set! action-procedures (cons proc action-procedures))
      (proc))
    
    ; 返回一个内部过程，有参数的。
    (define (dispatch m)
      (cond ((eq? m 'get-signal) signal-value)
            ((eq? m 'set-signal!) set-my-signal!)
            ((eq? m 'add-action!) accept-action-procedure!)
            (else (error "Unknown operation -- WIRE" m))))
    dispatch))

; 这个call-each是一个诸葛调用一个无参数过程表中的过程的
(define (call-each procedures)
  (if (null? procedures)               ; 递归有终结条件。
      'done
      (begin
       ((car procedures))              ; 调用这个过程。
       (call-each (cdr procedures))))) ; 递归。

; 如下是简写
(define (get-signal wire)
 (wire 'get-signal))
(define (set-signal! wire new-value)
 ((wire 'set-signal!) new-value))
(define (add-action! wire action-procedure)
 ((wire 'add-action!) action-procedure))

; 总结一下，首先假设有线路，

总结一下。
- 第一轮
   - 首先假设有线路的构造过程。
   - 假设有了最基本门电路。
   - 然后在门电路的基础上做半加器
   - 然后在半加器的基础上做全加器。
- 第二轮
   - 假设我么有线路的几个功能。
      - get-signal : 返回连接线上的当前值。
      - set-signal! : 修改成新的值
      - add-action  : 添加信号值改变后需要运行的过程。
   - 根据如上的，就可以构建如下门电路
      - 与门
      - 非门
      - 或门
- 第三轮
   - 具体写线路
      - 构造过程
      - call-each 运行每个动作过程。
   - 待处理表 ： 其实就是等待处理的过程。
   - 写驱动，对待处理表操作，
- 第四轮
   - 加一个检测器放在线路上，用于显示模拟器的活动。

## 约束的传播

我们可以基于各种关系进行工作，这个语言的基本元素就是基本约束，它们描述了在不同量之间的某种特定关系。这里以温度转换为例子。

In [3]:
; 简单点，这个是有连接器（相当于电线），约束器（相当于电路）
; 如下是连接器的构造过程，我已经按照顺序写的，实际上，文章中是先从顶层开始抽象设计。
(define (make-connector)
  (let 
    ((value false)         ; 保存连接器的当前值
     (informant false)     ; 是设置连接器值的对象
     (constraints '()))    ; 这一连接器所涉及的所有约束的表。
    ; 如下这个修改新的值
    (define (set-my-value newval setter)
      (cond ((not (has-value? me))      ; 原先没有值的情况，
             (set! value newval)
             (set! informant setter)
             (for-each-except setter
                              inform-about-value
                              constraints))
            ((not (= value newval))
             (error "Contradiction" (list value newval)))
            (else 'ignored)))
    ; 如下是忘记原先的值
    (define (forget-my-value retractor)
      (if (eq? retractor informant)
          (begin (set! informant false)
                 (for-each-except retractor
                                  inform-about-no-value
                                  constraints))
          'ignored))
    ; 添加新的约束器
    (define (connect new-constraint)
      (if (not (memq new-constraint constraints))
          (set! constraints
                (cons new-constraint constraints)))
      (if (has-value? me)
          (inform-about-value new-constraint))
      'done)
    ; 如下这个是分发过程。
    (define (me request)
      (cond ((eq? request 'has-value?)
             (if informant true false))
            ((eq? request 'value) value)
            ((eq? request 'set-value!) set-my-value)
            ((eq? request 'forget) forget-my-value)
            ((eq? request 'connect) connect)
            (else (error "Unknown operation -- CONNECTOR"
                         request))))
    me))
; 连接器通知它的所有约束。
(define (for-each-except exception procedure list)
  (define (loop items)
    (cond ((null? items) 'done)
          ((eq? (car items) exception) (loop (cdr items)))   ; 这个约束会跳过通知过来的约束。
          (else (procedure (car items))
                (loop (cdr items)))))
  (loop list))

; 如下是连接器的外部界面
(define (has-value? connector)
  (connector 'has-value?))
(define (get-value connector)
  (connector 'value))
(define (set-value! connector new-value informant)
  ((connector 'set-value!) new-value informant))
(define (forget-value! connector retractor)
 ((connector 'forget) retractor))
(define (connect connector new-constraint)
 ((connector 'connect) new-constraint))

; 如下是构造几个约束器
(define (adder a1 a2 sum)
  ; 更新新值的过程
  (define (process-new-value)
    ; 判断2个值，然后计算第三个值
    (cond ((and (has-value? a1) (has-value? a2))
           (set-value! sum
                       (+ (get-value a1) (get-value a2))
                       me))
          ((and (has-value? a1) (has-value? sum))
           (set-value! a2
                       (- (get-value sum) (get-value a1))
                       me))
          ((and (has-value? a2) (has-value? sum))
           (set-value! a1
                       (- (get-value sum) (get-value a2))
                       me))))
  ; 忘记值
  (define (process-forget-value)
    (forget-value! sum me)   ; 如下是3个忘记值，会导致所有相关的值忘记。
    (forget-value! a1 me)
    (forget-value! a2 me)
    (process-new-value))     ; 计算新的值
 (define (me request)
   (cond ((eq? request 'I-have-a-value)
          (process-new-value))
         ((eq? request 'I-lost-my-value)
          (process-forget-value))
         (else
          (error "Unknown request -- ADDER" request))))
  (connect a1 me)
  (connect a2 me)
  (connect sum me)
  me)

; 外部界面
(define (inform-about-value constraint)
  (constraint 'I-have-a-value))
(define (inform-about-no-value constraint)
  (constraint 'I-lost-my-value))

; 乘法器约束
(define (multiplier m1 m2 product)
  (define (process-new-value)
    (cond ((or (and (has-value? m1) (= (get-value m1) 0))
               (and (has-value? m2) (= (get-value m2) 0)))
           (set-value! product 0 me))
          ((and (has-value? m1) (has-value? m2))
           (set-value! product
                       (* (get-value m1) (get-value m2))
                       me))
          ((and (has-value? product) (has-value? m1))
           (set-value! m2
                       (/ (get-value product) (get-value m1))
                       me))
          ((and (has-value? product) (has-value? m2))
           (set-value! m1
                       (/ (get-value product) (get-value m2))
                       me))))
  (define (process-forget-value)
    (forget-value! product me)
    (forget-value! m1 me)
    (forget-value! m2 me)
    (process-new-value))
  (define (me request)
    (cond ((eq? request 'I-have-a-value)
           (process-new-value))
          ((eq? request 'I-lost-my-value)
           (process-forget-value))
          (else
           (error "Unknown request -- MULTIPLIER" request))))
  (connect m1 me)
  (connect m2 me)
  (connect product me)
  me)
; 常量约束
(define (constant value connector)
  (define (me request)
    (error "Unknown request -- CONSTANT" request))
  (connect connector me)
  (set-value! connector value me)
  me)

; 监视器
(define (probe name connector)
  (define (print-probe value)
    (newline)
    (display "Probe: ")
    (display name)
    (display " = ")
    (display value))
  (define (process-new-value)
    (print-probe (get-value connector)))
  (define (process-forget-value)
    (print-probe "?"))
  (define (me request)
    (cond ((eq? request 'I-have-a-value)
           (process-new-value))
          ((eq? request 'I-lost-my-value)
           (process-forget-value))
          (else
           (error "Unknown request -- PROBE" request))))
  (connect connector me)
  me)

; 如下是构造温度


(define (celsius-fahrenheit-converter c f)
  (let 
    ; 临时变量
    ((u (make-connector))
     (v (make-connector))
     (w (make-connector))
     (x (make-connector))
     (y (make-connector)))
    ; 构造完整约束
    (multiplier c w u)
    (multiplier v x u)
    (adder v y f)
    (constant 9 w)
    (constant 5 x)
    (constant 32 y)
    'ok))

(define C (make-connector))
(define F (make-connector))
(celsius-fahrenheit-converter C F)


'ok

In [4]:
; 添加监视器
(probe "Celsius temp" C)
(probe "Fahrenheit temp" F)

#<procedure>

In [5]:
(set-value! C 25 'user)



Probe: Celsius temp = 25
Probe: Fahrenheit temp = 77

'done

In [6]:
(set-value! F 212 'user)

Contradiction {77 212}


In [7]:
(forget-value! C 'user)


Probe: Celsius temp = ?
Probe: Fahrenheit temp = ?

'done

In [8]:
(set-value! F 212 'user)


Probe: Fahrenheit temp = 212
Probe: Celsius temp = 100

'done

总结一下：
- 三大部分：
   - 连接线部分 ： 里边存储约束器，当有变化时，通知约束器。
   - 约束器部分 ： 当有足够的信息支持改变数据的时候，修改相关的系数，而这个修改或者删除过程会调用连接线部分的过程，从而继续。
   - 监听器部分 ： 其实跟约束器一样，只是监听数据而已。

In [9]:
; 联系3.33 c=(a+b)/2
; 这里我不做除法，而当作乘法吧
; c = (a+b) * 1/2

; 简单点，首先做3个连接
(define a (make-connector))
(define b (make-connector))
(define c (make-connector))
; 然后a+b为
(define ab (make-connector))
; 然后1/2是如下
(define ab-half (make-connector))

; 如下是做约束了
(adder a b ab)               ; ab = a + b
(multiplier ab ab-half c)    ; ab * ab-half = c
(constant 1/2 ab-half)       ; ab-half = 1/2

; 添加监听器
(probe 'a: a)
(probe 'b: b)
(probe 'c: c)

; 测试
(set-value! a 25 'user)
(set-value! b 35 'user)
(get-value c)


Probe: a: = 25
Probe: b: = 35
Probe: c: = 30

30

# 并行化

In [10]:
; 如下检查单元结果，但这个并不能保证达到所需要的效果，这个操作必须以原子擦做
; 如下的为了照顾racket，做了很多修改，比如car->mcar , list->mcons,make-mutex中的if缺少else。
(require racket)
;; (define false 0)
;; (define true 1)
(define (test-and-set! cell)
  (if (mcar cell)
      true                            ; 如果为真，表示别的进程有互斥元               
      (begin (set-car! cell true)     ; 这里表示可以获得互斥元，先将互斥元设置为真
             false)))                 ; 然后再返回假。   

(define (make-serializer)       ; 串行化组定义
  (let ((mutex (make-mutex)))   ; 这个是互斥元
    (lambda (p)                 ; 串行化组的参数是过程
      (define (serialized-p . args)
        (mutex 'acquire)             ; 先获得互斥元
        (let ((val (apply p args)))  ; 执行过程，并保存结果。
          (mutex 'release)           ; 释放互斥元。
          val))                      ; 返回结果    
      serialized-p)))


(define (make-mutex)                    ; 构造互斥元过程
  (let ((cell (mcons false false)))
    (define (the-mutex m)               ; 分发消息
      (cond ((eq? m 'acquire)           ; 如果是要获得互斥元
             (if (test-and-set! cell)   ; 检查单元
                 (the-mutex 'acquire)
                 nil
                 )) ; retry
            ((eq? m 'release) (clear! cell))))  ; 释放互斥元
    the-mutex))

(define (clear! cell)
 (set-car! cell false))


(define (parallel-execute . args) (for-each thread args))  ; 合格是一种并行。

(define x 10)
(define s (make-serializer))     ; 这个是做一个串行化组
(parallel-execute 
 (s (lambda () (set! x (* x x))))
 (s (lambda () (set! x (+ x 1)))))
; 我要打印100次
(define (manycount var count)
  (if (= count 0)
      nil
      (begin (display x)
             (display ",")
             (manycount var (- count 1)))))
(manycount x 200)  ; 我电脑运行太快，这个不运行也正常。

11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,11,

'()

串行化的实现： 用一种更基本的称为互斥元的同步机制来实现串行化。 互斥元是一种对象，假设提供2个操作，一个互斥元可以被获取和被释放，一旦某个互斥元被获取，对于这一互斥元的任何其他操作都必须等到该互斥元被释放之后。

In [11]:
(define cell (list false))

In [12]:
(if (car cell)
    (display 'true)
    (display 'false))

false

In [13]:
(require racket)
(define x 10)
(define (parallel-execute . thunks)
  (for-each thread thunks))
(parallel-execute (lambda () (set! x (* x x)))
                  (lambda () (set! x (+ x 1))))
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)
(newline)
(display x)

10
10
10
10
10
10
10
10
10
10
10
11
11
1111
11
11
11
11
11
11
11
11
11
11
121
121
121

# 流

In [14]:
; 序列 ： 一次操作完全执行完毕后才发给下一层，如果数据很大，那么中间数据会非常巨大。
; 流 ： 一次部分的提供数据给下一层，这样不就增加了效率吗？

; 序列是分3层次，1、序列(list)，2、产生序列的过程,3、序列的高阶函数，4、操作序列的单个项目的过程（比如筛选）

; 流的实现：
; 1、 用序对，序对的第一项先计算出来，然后序对的第二项是将来要计算的
; (cons-stream <a> <b>)  
; 等价于
; (cons <a> (delay <b>))   ; delay是约定以后要计算的，现在不计算。
; (define (stream-car stream) (car stream))
; (define (stream-cdr stream) (force (cdr stream)))   ; force 相当于要执行参数过程。
; 2、 产生序列的过程中返回 (cons-stream <a> <b>) ，这样，其实只是计算了第一个<a>，<b>是约定以后计算但还没有计算的。
; 3、 高阶函数中，
(define (stream-filter pred stream)
  (cond ((stream-null? stream) the-empty-stream)     ; 如果为空。
        ((pred (stream-car stream))                  ; 这里是操作<a>
         (cons-stream (stream-car stream)            ; 这个不是筛选吗？上边的为真后，这里继续生成一个cons-stream给上一层。
                      (stream-filter pred
                                     (stream-cdr stream))))
        (else (stream-filter pred (stream-cdr stream)))))  ; 这里迫使计算<b>,组合式求值，先求子表达式。
; 4、 delay的实现
; (delay <exp>) 实际上是 (lambda () <exp>) 
; 而force 是
; (define (force delayed-object)
;  (delayed-object))     ; 只是将参数当作过程执行。
; 总结 ： 序列，是一次全部求出所有的数据，然后送给下一层，而流，则是一次求出部分数据给下一层。

; 保存求值的。
(define (memo-proc proc)
  (let ((already-run? false) (result false))
    (lambda ()
      (if (not already-run?)                ; 如果没有数据
          (begin (set! result (proc))       ; 
                 (set! already-run? true)
                 result)
          result))))                        ; 如果已经有结果就返回结果
; 这样 (delay <exp>)  等价于 (memo-proc (lambda () <exp>))